Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB

Let's try to make a Gaussian node with several different sizes and parameterizations:

In [2]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

3

In [3]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [4]:
μ[1]

3-element Array{Float64,1}:
 0.124232
 0.20371 
 0.213106

In [5]:
Σ[1]

3x3 Array{Float64,2}:
 1.49543   1.20489   0.953785
 1.20489   1.15408   0.838758
 0.953785  0.838758  0.638243

In [6]:
det(Σ[2])

0.02126852628725841

In [7]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...)

LoadError: LoadError: UndefVarError: MultivariateNormal not defined
while loading In[7], in expression starting on line 2

In [8]:
rr = RandomNode(aa, Factor[]);

In [9]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [10]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [11]:
ss = RandomNode(MultivariateNormal, pars...);

In [12]:
E(ss)

5x5 Array{Array{Float64,1},2}:
 [0.654580803585662,0.002082610838291199,0.46904245725046945]  …  [0.2873768581650793,0.6641040963768385,0.950732530779683]   
 [0.04149202311334266,0.9208919227014258,0.9669079688025559]      [0.013508199068207327,0.7109722567114416,0.6920752443361871]
 [0.6357933951330921,0.4788095987493908,0.8595479973621487]       [0.2716502042236484,0.6828627487985028,0.018648969519561076]
 [0.10702062862787032,0.49155223402422155,0.3586245163943229]     [0.5900206142682802,0.20775631188197363,0.13093060230215348]
 [0.8368950681274876,0.6478403061283988,0.6371474485902417]       [0.7034115931075446,0.15449806787263998,0.5087403968761628] 

In [13]:
?E

search: 

Calculates the expected value of a Node x.


E e eu ENV end Expr eye exp eta esc erf eps eof eig Enum else expm exp2



In [20]:
typeof(ss).parameters[1]

Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}}